In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Rapid Eval SDK GA - Define Your Own Metrics

{TODO: Update the links below. Use the next code cell to generate the HTML, then delete the code cell.}

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fnotebook_template.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/notebook_template.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) [Jason Dai](https://github.com/jsondai) [Bo Zheng](https://github.com/coolalexzb) [Naveksha Sood](https://github.com/navekshasood) |

## Overview

Use Rapid Evaluation SDK to evaluate multiple prompt templates with your custom defined metrics.

## Get started

### Install Vertex AI SDK and other required packages


In [ ]:
!pip3 install --upgrade --user --quiet google-cloud-aiplatform[evaluation]

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

import vertexai

PROJECT_ID = "cloud-llm-preview1"  # @param {type:"string", isTemplate: true}
if PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [ ]:
# General
import inspect
from uuid import uuid4
from google.colab import auth
from IPython.display import display, Markdown, HTML
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import logging
import random
import string
import warnings

# Main
import pandas as pd
from google.cloud import aiplatform
from vertexai.generative_models import GenerativeModel, HarmCategory, HarmBlockThreshold

from vertexai.evaluation import (
    EvalTask,
    CustomMetric,
    PointwiseMetric,
    PairwiseMetric,
    PointwiseMetricPromptTemplate,
    PairwiseMetricPromptTemplate,
    MetricPromptTemplateExamples,
)

In [ ]:
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

### Helper Functions

In [ ]:
def print_doc(function):
    """Print the docstring of a function."""
    print(f"{function.__name__}:\n{inspect.getdoc(function)}\n")

def display_explanations(eval_result, metrics=None, n=1):
    """Display the explanations."""
    style = "white-space: pre-wrap; width: 1500px; overflow-x: auto;"
    metrics_table = eval_result.metrics_table
    df = metrics_table.sample(n=n)

    if metrics:
          df = df.filter(
              ['response', 'baseline_model_response'] +
              [metric for metric in df.columns if any(selected_metric in metric for selected_metric in metrics)]
          )
    for index, row in df.iterrows():
      for col in df.columns:
        display(
            HTML(
                f"<div style='{style}'><h4>{col}:</h4>{row[col]}</div>"
            )
        )
      display(HTML("<hr>"))

def display_eval_result(eval_result, title=None, metrics = None):
    """Display the evaluation results."""
    summary_metrics, metrics_table = eval_result.summary_metrics, eval_result.metrics_table

    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient='index').T
    if metrics:
      metrics_df = metrics_df.filter([metric for metric in metrics_df.columns if any(selected_metric in metric for selected_metric in metrics)])
      metrics_table = metrics_table.filter([metric for metric in metrics_table.columns if any(selected_metric in metric for selected_metric in metrics)])

    if title:
      # Display the title with Markdown for emphasis
      display(Markdown(f"## {title}"))
    # Display the summary metrics DataFrame
    display(Markdown("### Summary Metrics"))
    display(metrics_df)
    # Display the metrics table DataFrame
    display(Markdown(f"### Row-based Metrics"))
    display(metrics_table)

def display_radar_plot(eval_results, metrics = None):
  """Plot the radar plot."""
  fig = go.Figure()
  for item in eval_results:
    eval_result, title = item
    summary_metrics = eval_result.summary_metrics
    if metrics:
      summary_metrics = {k.replace('/mean',''): summary_metrics[k] for k, v in summary_metrics.items() if any(selected_metric+'/mean' in k for selected_metric in metrics)}
    fig.add_trace(go.Scatterpolar(
          r=list(summary_metrics.values()),
          theta=list(summary_metrics.keys()),
          fill='toself',
          name=title
    ))
  fig.update_layout(
    polar=dict(
      radialaxis=dict(
        visible=True,
        range=[0, 5]
      )),
    showlegend=True
  )
  fig.show()

def display_bar_plot(eval_results_list, metrics=None):

  fig = go.Figure()
  data = []

  for eval_results in eval_results_list:
    eval_result, title = eval_results[0], eval_results[1]

    summary_metrics = eval_result.summary_metrics
    mean_summary_metrics = [f"{metric}/mean" for metric in metrics]
    updated_summary_metrics = []
    if metrics:
      for k, v in summary_metrics.items():
        if k in mean_summary_metrics:
          updated_summary_metrics.append((k, v))
      summary_metrics = dict(updated_summary_metrics)
      # summary_metrics = {k: summary_metrics[k] for k, v in summary_metrics.items() if any(selected_metric in k for selected_metric in metrics)}

    data.append(go.Bar(
          x=list(summary_metrics.keys()),
          y=list(summary_metrics.values()),
          name=title,
    ))

  fig = go.Figure(data=data)

  # Change the bar mode
  fig.update_layout(barmode='group', showlegend=True)
  fig.show()

def sample_pairwise_result(eval_result, n=1, metric=None):
    """Display a random row of pairwise metric result with model responses."""
    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = eval_result.metrics_table
    df = df.sample(n=n)
    for index, row in df.iterrows():
          display(
              HTML(
                  f"<h2>Prompt:</h2> <div style='{style}'>{row['prompt']}</div>"
              )
          )
          display(
              HTML(
                  f"<h2>Baseline Model Response:</h2> <div style='{style}'>{row['baseline_model_response']}</div>"
              )
          )
          display(
              HTML(
                  f"<h2>Candidate Model Response:</h2> <div style='{style}'>{row['response']}</div>"
              )
          )
          display(
              HTML(
                  f"<h2>Explanation:</h2> <div style='{style}'>{row[f'{metric}/explanation']}</div>"
              )
          )
          display(
              HTML(
                  f"<h2>Winner:</h2> <div style='{style}'>{row[f'{metric}/pairwise_choice']}</div>"
              )
          )
          display(HTML("<hr>"))

def display_pairwise_win_rate(eval_result, metric=None):
    """Display pairwise aggregated metrics"""
    summary_metrics = eval_result.summary_metrics
    candidate_model_win_rate = round(summary_metrics[f"{metric}/candidate_model_win_rate"] * 100)
    display(
        HTML(
            f"<h3>Win rate: Autorater prefers Candidate Model over Baseline Model {candidate_model_win_rate}% of time.</h3>"
        )
    )

def generate_uuid(length: int = 8) -> str:
    """Generate a uuid of a specified length (default=8)."""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))

## Define an Evaluation Dataset

In [ ]:
instruction = 'Summarize the following article: \n'

context = [
    'Typhoon Phanfone has killed at least one person, a US airman on Okinawa who was washed away by high waves. Thousands of households have lost power and Japan\'s two largest airlines have suspended many flights. The storm also forced the suspension of the search for people missing after last week\'s volcanic eruption. The storm-tracking website Tropical Storm Risk forecasts that Phanfone will rapidly lose power over the next few hours as it goes further into the Pacific Ocean. Typhoon Phanfone was downgraded from an earlier status of a super typhoon, but the Japan Meteorological Agency had warned it was still a dangerous storm. Japan averages 11 typhoons a year, according to its weather agency. The typhoon made landfall on Monday morning near the central city of Hamamatsu, with winds of up to 180 km/h (112 mph). The airman was one of three US military personnel swept away by high waves whipped up by the typhoon off southern Okinawa island, where the US has a large military base. The remaining two are still missing. A police spokesman said they had been taking photographs of the sea. A university student who was surfing off the seas of Kanagawa Prefecture, south of Tokyo, was also missing, national broadcast NHK reports. It said at least 10 people had been injured and 9,500 houses were without power. The storm was expected to deposit about 100mm of rain on Tokyo over 24 hours, according to the Transport Ministry website. Many schools were closed on Monday and two car companies in Japan halted production at some plants ahead of the storm. More than 174 domestic flights were affected nationwide, NHK state broadcaster said on Sunday. On Sunday, heavy rain delayed the Japanese Formula One Grand Prix in Suzaka. French driver Jules Bianchi lost control in the wet conditions and crashed, sustaining a severe head injury.',
    'The blaze started at the detached building in Drivers End in Codicote, near Welwyn, during the morning. There was another fire at the building 20 years ago, after which fire-proof foil was placed under the thatch, which is protecting the main building. More than 15 fire engines and support vehicles were called to tackle the blaze. Roads in the area were closed and traffic diverted.',
    'The 18-year-old fell at the New Charter Academy on Broadoak Road in Ashton-under-Lyne at about 09:10 BST, Greater Manchester Police (GMP) said. GMP said he had gone to Manchester Royal Infirmary and his condition was "serious". Principal Jenny Langley said the school would remain "fully open" while police investigated. "Our thoughts are with the family and we\'re doing everything we can to support them along with staff and pupils," she said.',
    'But Belgian-born Dutchman Max Verstappen was unable to drive a car legally on his own in either country. That all changed on Wednesday when the youngster turned 18 and passed his driving test at the first attempt. Despite having competed in 14 grands prix since his debut in Australia in March, Verstappen admitted to feeling the pressure during his test. "It\'s a relief," said the Toro Rosso driver, who finished ninth in Japan on Sunday and had only started driving lessons a week ago. "I was a bit nervous to make mistakes, but the exam went well." A bonus of turning 18 is that Verstappen will now be able to drink the champagne if he ever makes it onto the podium.',
]

reference = [
    'A powerful typhoon has brought many parts of Japan to a standstill and briefly battered Tokyo before heading out to sea.',
    'A major fire has been burning in the thatched roof of a large property in Hertfordshire.',
    'A student has been taken to hospital after falling from a balcony at a Greater Manchester school.',
    'He is Formula 1\'s youngest ever driver and in charge of a car that can reach over 200mph.',
]

response = [
    'Typhoon Phanfone, while downgraded from super typhoon status, caused significant disruption and tragedy in Japan. One US airman died after being swept away by high waves, with two more missing. The storm caused power outages for thousands, flight cancellations, and the suspension of rescue efforts for missing volcano victims. Heavy rain and strong winds led to school and factory closures, transportation disruptions, and at least 10 injuries. The typhoon is expected to weaken as it moves over the Pacific Ocean.',
    'A large fire broke out in a detached thatched building in Codicote, near Welwyn. This is the second fire at the building in 20 years. Thankfully, fire-proof foil installed after the previous fire is protecting the main building. Over 15 fire engines and support vehicles responded, closing roads and diverting traffic in the area.',
    "An 18-year-old student at New Charter Academy in Ashton-under-Lyne suffered a serious fall and was hospitalized. The incident is under investigation by Greater Manchester Police, but the school remains open. The principal expressed support for the student's family and the school community.",
    "Max Verstappen, a Formula One driver, was finally able to get his driver's license at age 18. Despite already competing in 14 Grand Prix races, he was not of legal driving age in his native countries. He admitted to being nervous but passed the test on his first attempt.  As an added bonus of turning 18, Verstappen can now enjoy champagne on the podium if he places."
]

eval_dataset = pd.DataFrame({
    "prompt"       : [instruction + item for item in context],
    "reference"    : reference,
})

## Define Your Own Metrics

### Templated Customization - Pointwise Metric

In [ ]:
criteria = {
    "Proper Grammar": "The language's grammar rules are correctly followed, including but not limited to sentence structures, verb tenses, subject-verb agreement, proper punctuation, and capitalization.",
    "Smooth": "Sentences flow smoothly and are easy to read, avoiding awkward phrasing or run-on sentences. Ideas and sentences connect logically, using transitions effectively where needed.",
    "Appropriate word choice": "Words chosen are appropriate and purposeful given their relative context and positioning in the text. Vocabulary demonstrates prompt understanding."
}

pointwise_rating_rubric = {
    "5": "Completly Coherent: The writing has a seamless logical flow, is expertly organized, and maintains exceptional cohesion throughout.",
    "4": "Mostly coherent: The writing demonstrates strong logical flow, a clear structure, and demonstrates good cohesion.",
    "3": "Somewhat coherent: The writing's logical flow is mostly understandable, it has a recognizable structure, and cohesion is present but could be stronger.",
    "2": "Somewhat incoherent: The writing lacks a clear logical flow, organizational structure is weak, and cohesion is inconsistent or confusing.",
    "1": "Completely incoherent: The writing is highly illogical, lacks any clear organization, and has little to no cohesion.",
}

In [ ]:
# The metric prompt template contains default prompts pre-defined for unspecified components.
linguistic_acceptability_metric_prompt_template = PointwiseMetricPromptTemplate(
    criteria=criteria,
    rating_rubric=pointwise_rating_rubric,
    input_variables=["prompt"],
)

# Display the assembled prompt template that will be sent to Gen AI Eval Service
# along with the input data for model-based evaluation.
print(linguistic_acceptability_metric_prompt_template.prompt_data)

In [ ]:
# Model to be evaluated
generation_config = {"temperature": 0.5, "max_output_tokens": 256, "top_k": 1}

model = GenerativeModel(
    'gemini-1.5-pro',
    generation_config=generation_config
)

In [ ]:
# Define this lcustom "inguistic_acceptability" model-based metric.
linguistic_acceptability = PointwiseMetric(
    metric="linguistic_acceptability",
    metric_prompt_template=linguistic_acceptability_metric_prompt_template,
)

linguistic_acceptability_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[linguistic_acceptability],
)

linguistic_acceptability_result = linguistic_acceptability_eval_task.evaluate(
    model=model,
)

In [ ]:
display_eval_result(linguistic_acceptability_result)

In [ ]:
display_explanations(linguistic_acceptability_result, metrics=['linguistic_acceptability'], n=1)

### Templated Customization - Pairwise Metric

In [ ]:
baseline_model = GenerativeModel(
  'gemini-1.0-pro',
)

In [ ]:
pairwise_rating_rubric = {
    "A": "Response A answers the given question as per the criteria better than response B.",
    "SAME": "Response A and B answers the given question equally well as per the criteria.",
    "B": "Response B answers the given question as per the criteria better than response A.",
}

pairwise_linguistic_acceptability_prompt_template = PairwiseMetricPromptTemplate(
    criteria=criteria,
    rating_rubric=pairwise_rating_rubric,
    input_variables=["prompt"],
)

pairwise_linguistic_acceptability = PairwiseMetric(
    metric="pairwise_linguistic_acceptability",
    metric_prompt_template=pairwise_linguistic_acceptability_prompt_template,
    baseline_model=baseline_model
)

print(pairwise_linguistic_acceptability_prompt_template.prompt_data)

In [ ]:
pairwise_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[pairwise_linguistic_acceptability],
)

pairwise_result = pairwise_eval_task.evaluate(
    model=model,
)

In [ ]:
display_eval_result(pairwise_result)

In [ ]:
sample_pairwise_result(pairwise_result, metric='pairwise_linguistic_acceptability')

### Free-form customization - Pointwise Metric

In [ ]:
free_form_pointwise_metric_prompt = """
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user prompt and an AI-generated responses.
You should first read the user prompt carefully for analyzing the task, and then evaluate the quality of the responses based on and Criteria provided in the Evaluation section below.

You will assign the response a score from 5, 4, 3, 2, 1, following the Rating Rubric and Evaluation Steps. Give step-by-step explanations for your scoring, and only choose scores from 5, 4, 3, 2, 1.

# Evaluation
## Metric Definition
You will be assessing Text Quality, which measures how effectively the text conveys clear, accurate, and engaging information that directly addresses the user's prompt, considering factors like fluency, coherence, relevance, and conciseness.

## Criteria
Coherence: The response presents ideas in a logical and organized manner, with clear transitions and a consistent focus, making it easy to follow and understand.
Fluency: The text flows smoothly and naturally, adhering to grammatical rules and using appropriate vocabulary.
Instruction following: The response demonstrates a clear understanding of the summarization task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context. The response does not reference any outside information.
Verbosity: The response is appropriately concise, providing sufficient detail without using complex language to thoroughly address the prompt without being overly wordy or excessively brief.

## Rating Rubric
5: (Very good). Exceptionally clear, coherent, fluent, and concise. Fully adheres to instructions and stays grounded.
4: (Good). Well-written, coherent, and fluent. Mostly adheres to instructions and stays grounded. Minor room for improvement.
3: (Ok). Adequate writing with decent coherence and fluency. Partially fulfills instructions and may contain minor ungrounded information. Could be more concise.
2: (Bad). Poorly written, lacking coherence and fluency. Struggles to adhere to instructions and may include ungrounded information. Issues with conciseness.
1: (Very bad). Very poorly written, incoherent, and non-fluent. Fails to follow instructions and contains substantial ungrounded information. Severely lacking in conciseness.

## Evaluation Steps
STEP 1: Assess the response in aspects of all criteria provided. Provide assessment according to each criterion.
STEP 2: Score based on the rating rubric. Give a brief rationale to explain your evaluation considering each individual criterion.

# User Inputs and AI-generated Response
## User Inputs
### Prompt
{prompt}

## AI-generated Response
{response}
"""

In [ ]:
free_form_pointwise_text_quality_metric = PointwiseMetric(
    metric="free_form_pointwise_text_quality",
    metric_prompt_template=free_form_pointwise_metric_prompt,
)


free_form_pointwise_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[free_form_pointwise_text_quality_metric],
)

free_form_pointwise_result = free_form_pointwise_eval_task.evaluate(
    model=model
)

In [ ]:
display_eval_result(free_form_pointwise_result)

### Free-form customization - Pairwise Metric

In [ ]:
free_form_pairwise_metric_prompt = """
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by two AI models. We will provide you with the user input and a pair of AI-generated responses (Response A and Response B).
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.
You will first judge responses individually, following the Rating Rubric and Evaluation Steps.
Then you will give step-by-step explanations for your judgement, compare results to declare the winner based on the Rating Rubric and Evaluation Steps.

# Evaluation
## Metric Definition
You will be assessing the verbosity of each model's response, which measures its conciseness and ability to provide sufficient detail without being overly wordy or excessively brief.

## Criteria
Verbosity: The response is appropriately concise, providing sufficient detail without using complex language to thoroughly address the prompt without being overly wordy  or excessively brief.

## Rating Rubric
"A": Response A is more appropriately concise than Response B. It strikes a better balance between providing sufficient detail and avoiding unnecessary wordiness or excessive brevity.
"SAME": Response A and B are equally concise. They both strike the same level of balance between providing sufficient detail and avoiding unnecessary wordiness or excessive brevity.
"B": Response B is more appropriately concise than Response A. It strikes a better balance between providing sufficient detail and avoiding unnecessary wordiness or excessive brevity.

## Evaluation Steps
STEP 1: Analyze Response A based on the Verbosity criterion regarding completeness, conciseness, and overall balance.
STEP 2: Analyze Response B based on the Verbosity criterion regarding completeness, conciseness, and overall balance.
STEP 3: Compare the overall performance of Response A and Response B based on your analyses and assessment.
STEP 4: Output your preference of "A", "SAME" or "B" to the pairwise_choice field according to the Rating Rubric.
STEP 5: Output your assessment reasoning in the explanation field, justifying your choice by highlighting the specific strengths and weaknesses of each response in terms of verbosity.

# User Inputs and AI-generated Responses
## User Inputs
### Prompt
{prompt}

# AI-generated Responses

### Response A
{baseline_model_response}

### Response B
{response}
"""

In [ ]:
free_form_pairwise_metric = PairwiseMetric(
    metric="free_form_pairwise_verbosity",
    metric_prompt_template=free_form_pairwise_metric_prompt,
    baseline_model=baseline_model
)
free_form_pairwise_eval_task = EvalTask(
  dataset=eval_dataset,
  metrics=[free_form_pairwise_metric],
)

free_form_pariwise_result = free_form_pairwise_eval_task.evaluate(
	model=model,
)

In [ ]:
display_eval_result(free_form_pariwise_result)